In this project, we will be using the LIBOR Forward Market Model(LFMM) to simulate the forward rates.

First, we import the necessary libraries needed for the code.

In [1]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import random

Next, we define the list of parameter values needed.

In [2]:
L = 150
S_0 = 100
K = S_0
V_0 = 200
D = 175
gamma = 0.75
sigma = 0.3
correlation = 0.2
recovery_rate = 0.25
n_simulations = 5
dt = 1/12

# Calibration

We are first going to calibrate the LFMM using the observed zero coupon bond prices. 

The jth forward rate at time t can be defined as

$F_j(t) = \frac{P_j(t)-P_{j+1}(t)}{\delta_t P_{j+1}(t)}$

We are going to use this formula to calibrate and obtain the initial forward rates, which will be used to simulate future rates as in the LFMM model.

In [3]:
def bondPricesToInitialForwardRates(bond_prices, n_steps):
    return np.ones([n_simulations, n_steps - 1]) * (bond_prices[:-1] - bond_prices[1:])/(dt * bond_prices[1:])

In [4]:
bond_prices = np.array([100, 99.38, 98.76, 98.15, 97.54, 96.94, 96.34, 95.74, 95.16, 94.57, 93.99, 93.42, 92.85])
n_steps = len(bond_prices) 

forward_rates = bondPricesToInitialForwardRates(bond_prices, n_steps)

# Interest Rate - LIBOR Forward Market Model (LFMM)

The LFMM says that the forward rate evolves according to the following formula

$F_j(t_i) = F_j(t_{i-1})exp[(\mu_j(t_{i-1})-\frac{1}{2}\sigma^2)\delta_{i-1} + \sigma_j\sqrt{\delta_{i-1}}Z_i]$

where

$\mu_j(t_{i-1}) \ \sum_{k=i}^j \frac{\delta_k F_j(t_{i-1})\sigma_k\sigma_j}{1+\delta_kF_j(t_{i-1})}$

which can be written as

In [5]:
def getMu(forward_rates, delta, sigma_j, sigma_k):
    return np.cumsum(delta * forward_rates * sigma_j * sigma_k/(1 + delta * forward_rates), axis=1)

def getInitialNextForwardEstimate(forward_rates, mu, sigma, delta, Z):
    return forward_rates * np.exp((mu - (sigma**2)/2) * delta + sigma * np.sqrt(delta) * Z)


However, to improve the accuracy, we will use the Predictor-Corrector method which involves estimating the next period's drift and then taking the average. Hence, the final estimate of the forward rate is given by:

$F_j(t_i) = F_j(t_{i-1})exp[\frac{1}{2}(\mu_j(t_{i-1})+\mu_j(t_i)-\sigma^2)\delta_{i-1} + \sigma_j\sqrt{\delta_{i-1}}Z_i]$

This final estimate is written as:

In [6]:
def getFinalNextForward(forward_rates, mu1, mu2, sigma, delta, Z):
    return forward_rates * np.exp(0.5 * (mu1 + mu2 - sigma**2) * delta + sigma * np.sqrt(delta) * Z)

Furthermore, the continuously compounded interest rate can be derived from the LIBOR forward rate as follows:

$e^{r_{ti}(t_{i+1}-t_i)} = 1 + L(t_i, t_{i+1})(t_{i+1} - t_i)$

In [7]:
def liborToContinuousRate(libor, dt):
    return 1 + (libor * dt)

# Volatility - CEV Model

We will be using the CEV model to represent volatility, which is given by

$\sigma(t_i, t_{i+1}) = \sigma(S_{it})^{\gamma-1}$

In [8]:
def getVolatility(S_t, gamma, sigma):
    return sigma * S_t**(gamma - 1)

# Stock Price Model

We assume the following lognormal evolution of stock price/firm value:

$S_{t_{i+1}} = S_{t_i}exp[(r - \frac{\sigma^2(t_i, t_{i+1})}{2})(t_{i+1}-t_i)+\sigma(t_i, t_{i+1})\sqrt{t_{i+1}-t_i}Z]$

which can be re-written as

$S_{t_{i+1}} = S_{t_i}exp[r(t_{i+1}-t_i)]exp[(- \frac{\sigma^2(t_i, t_{i+1})}{2})(t_{i+1}-t_i)+\sigma(t_i, t_{i+1})\sqrt{t_{i+1}-t_i}Z]$

In [9]:
def getSharePrice(S_t, continuous_rate, sigma, dt, Z):
    return S_t * continuous_rate * np.exp(-sigma**2/2 * dt + sigma * np.sqrt(dt) * Z)

# Simulation

First, we initialise the stock matrix and delta matrix.

In [10]:
delta = np.ones([n_simulations, n_steps - 1]) * dt
stock = np.ones([n_simulations, n_steps]) * S_0

We also need to define a method to reshape our data structure

In [11]:
def reshape(input):
    return np.array([input, ]).transpose()

Then, we go to the actual simulation.

In [12]:
print("Values for 1st Simulation only\n")

for i in range(0, 12):
    #each iteration is a point in time, estimating for 12 forward rates
    forward_Z = norm.rvs(size = [n_simulations, 1])
    stock_Z = norm.rvs(size = [n_simulations, 1])
    
    S_t_minus_one = stock[:, i]
    forward_rates_t = forward_rates[:, i:]
    delta_t = delta[:, i:]
    
    sigma_new = getVolatility(S_t_minus_one, gamma, sigma)
    #duplicate the sigma across more columns as the same sigma will be used to calculate the other forward rates 
    sigma_new_transposed = np.array([sigma_new,] * (n_steps - (i + 1))).transpose() 
    
    mu_initial = getMu(forward_rates_t, delta_t, sigma_new_transposed, sigma_new_transposed)
    forward_temp = getInitialNextForwardEstimate(forward_rates_t, mu_initial, sigma_new_transposed, delta_t, forward_Z)
    mu_next = getMu(forward_temp, delta_t, sigma_new_transposed, sigma_new_transposed)
    final_forwards = getFinalNextForward(forward_rates_t, mu_initial, mu_next, sigma_new_transposed, delta_t, forward_Z)
    forward_rates[:,i:] = final_forwards
    
    first_libor_rate = forward_rates[:,i]
    continuous_rate = liborToContinuousRate(first_libor_rate, dt)
    
    continuous_rate = reshape(continuous_rate)
    sigma_new = reshape(sigma_new)
    S_t_minus_one = reshape(S_t_minus_one)
    S_t = getSharePrice(S_t_minus_one, continuous_rate, sigma_new, dt, stock_Z)
    stock[:, i + 1] = S_t[:,0]
    
    print("Volatility at time " + str(i+1) + ": " + str(sigma_new[0]))
    print("Forward Rates at time " + str(i+1) + ": " + str(final_forwards[0]))
    print("First LIBOR rate at time " + str(i+1) + ": " + str(first_libor_rate[0]))
    print("Continuos rate at time " + str(i+1) + ": " + str(continuous_rate[0]))
    print("Stock Price at time " + str(i+1) + ": " + str(S_t[0]))
    print("\n")

Values for 1st Simulation only

Volatility at time 1: [0.09486833]
Forward Rates at time 1: [0.07183979 0.07229112 0.0715675  0.0720154  0.07127357 0.07171778
 0.07216756 0.07018749 0.07184338 0.07106184 0.07026305 0.07069471]
First LIBOR rate at time 1: 0.07183978736279592
Continuos rate at time 1: [1.00598665]
Stock Price at time 1: [100.22689313]


Volatility at time 2: [0.09481459]
Forward Rates at time 2: [0.07071908 0.0700115  0.07044997 0.06972457 0.07015944 0.07059976
 0.068663   0.07028324 0.06951897 0.06873782 0.06916041]
First LIBOR rate at time 2: 0.07071907699949354
Continuos rate at time 2: [1.00589326]
Stock Price at time 2: [104.09272946]


Volatility at time 3: [0.09392175]
Forward Rates at time 3: [0.07102826 0.07147341 0.07073777 0.07117927 0.0716263  0.06966167
 0.07130578 0.07053069 0.06973847 0.07016751]
First LIBOR rate at time 3: 0.07102825869520318
Continuos rate at time 3: [1.00591902]
Stock Price at time 3: [107.60455729]


Volatility at time 4: [0.09314587]
